In [1]:
from pulp import *
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Input data
heat_demand = pd.read_csv('heat_demand.csv', index_col=False)
el_demand = pd.read_csv('electric_demand.csv', index_col=False)
el_prices = pd.read_csv('electric_prices.csv', index_col=False)
eon_elprice = pd.read_csv('eon_elprice.csv', index_col=False)

In [14]:
# Input data
heat_demand = pd.read_csv('heat_demand.csv', index_col=False)
el_demand = pd.read_csv('electric_demand.csv', index_col=False)
el_prices = pd.read_csv('electric_prices.csv', index_col=False)
eon_elprice = pd.read_csv('eon_elprice.csv', index_col=False)

# Resource data
resources_heat = {
   
    'CHP_heat': {
        'min': 6,
        'max': 18.5,
        'efficiency': 1,
        'cost': 195.4       
         },
    'WC1_boiler': {
        'min': 4,
        'max': 9,
        'efficiency': 0.85,
        'cost': 195.4
    },
    'bio_oil_boiler_1': {
        'min': 0,
        'max': 6,
        'efficiency': 0.76,
        'cost': 1548.6
    },
    'bio_oil_boiler_2': {
        'min': 0,
        'max': 6,
        'efficiency': 0.76,
        'cost': 1548.6
    },
    'wood_pellets_boiler_1': {
        'min': 0.5,
        'max': 3.25,
        'efficiency': 0.87,
        'cost': 455.2 
    },
    'wood_pellets_boiler_2': {
        'min': 0.5,
        'max': 3.25,
        'efficiency': 0.87,
        'cost': 455.2 
    },
    'bio_oil_boiler_3': {
        'min': 0,
        'max': 10,
        'efficiency': 0.875,
        'cost': 1548.6
    }
}

HP={   
        'min': 0.1,
        'max': 5,
        'COP': 3
           }

resources_el = {
  'CHP_el': {
        'min': 0.8,
        'max': 3.9,
        'efficiency': 0.4,
        'cost': 248.5
    }}

Network_losses=0.091 # 9.1%

val=1000


# Initialize the optimization problem....................................................................................................

prob = LpProblem("District Heating Plant Dispatch Optimization", LpMinimize)


# Initialize the decision variables....................................................................................................

variables = LpVariable.dicts("Fuel", ((r, t) for r in resources_heat for t in heat_demand.index), lowBound=0, cat='Continuous')
b_heat = LpVariable("b_heat",((r, t) for r in resources_heat for t in heat_demand.index), cat="Binary")


      
# Objective function.............................................................................................

prob += lpSum([variables[(r, t)] * resources_heat[r]['cost'] for r in resources_heat for t in heat_demand.index])




# Constraints....................................................................................................
for t in heat_demand.index:
    # Heat demand constraint
    heat_production = lpSum([variables[(r, t)]  for r in resources_heat]) 
    
    
    prob += heat_production >= (1+Network_losses)*heat_demand.loc[t] 
    
    # Resource capacity constraints
    for r in resources_heat:
            prob += variables[(r, t)] <= resources_heat[r]['max']

    # Resource minimum load constraints
            
            #prob += b_heat >= variables[(r, t)]/val
            prob += variables[(r, t)] >= resources_heat[r]['min']*b_heat
    
    
    
    # Solve the optimization problem
prob.solve()


# Print the status of the solution
print("Status:", LpStatus[prob.status])

# Print the optimal value of the objective function
#print("Optimal Value: ", value(prob.objective))

# Print the values of the decision variables
for v in prob.variables():
    print(v.name == 'b_heat', "=", v.varValue)


# Define decision variables
chp_heat = {}
WC1 = {}
bio_oil_1 = {}
bio_oil_2 = {}
wood_pellets_1 = {}
wood_pellets_2 = {}

# Populate the decision variables with the optimal values
for i in heat_demand.index:
    chp_heat[i] = variables[('CHP_heat', i)]
    WC1[i] = variables[('WC1_boiler', i)]
    bio_oil_1[i] = variables[('bio_oil_boiler_1', i)]
    bio_oil_2[i] = variables[('bio_oil_boiler_2', i)]
    wood_pellets_1[i] = variables[('wood_pellets_boiler_1', i)]
    wood_pellets_2[i] = variables[('wood_pellets_boiler_2', i)]

# Populate the results DataFrame with the optimal values of the decision variables
results = pd.DataFrame(index=heat_demand.index, columns=['CHP_heat','WC1_boiler', 'Bio_oil_1', 'Bio_oil_2', 'wood_pellets_boiler_1', 'wood_pellets_boiler_2'])

for i in heat_demand.index:
    results.loc[i, 'CHP_heat'] = chp_heat[i].varValue
    results.loc[i, 'WC1_boiler'] = WC1[i].varValue
    results.loc[i, 'Bio_oil_1'] = bio_oil_1[i].varValue
    results.loc[i, 'Bio_oil_2'] = bio_oil_2[i].varValue
    results.loc[i, 'Bio_oil_3'] = bio_oil_2[i].varValue
    results.loc[i, 'wood_pellets_boiler_1'] = wood_pellets_1[i].varValue
    results.loc[i, 'wood_pellets_boiler_2'] = wood_pellets_2[i].varValue

# Print the results
#print(results)

results.to_excel("results.xlsx", index=False)

/Users/raul/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


TypeError: unsupported operand type(s) for /: 'LpVariable' and 'int'

In [7]:
results

,CHP_heat,WC1_boiler,Bio_oil_1,Bio_oil_2,wood_pellets_boiler_1,wood_pellets_boiler_2,Bio_oil_3
0,9.362043,9.0,0.0,0.0,0.0,0.0,0.0
1,18.5,0.412485,0.0,0.0,0.0,0.0,0.0
2,18.5,0.947751,0.0,0.0,0.0,0.0,0.0
3,18.5,0.802517,0.0,0.0,0.0,0.0,0.0
4,18.5,0.773704,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
8755,18.5,0.973161,0.0,0.0,0.0,0.0,0.0
8756,18.5,0.102401,0.0,0.0,0.0,0.0,0.0
8757,10.016806,9.0,0.0,0.0,0.0,0.0,0.0
8758,18.392383,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
heat_demand*(1+Network_losses)